https://learn.microsoft.com/en-us/azure/cosmos-db/nosql/tutorial-create-notebook-vscode?tabs=csharp

In [5]:
#r "nuget: Newtonsoft.Json, 13.0.3"
#r "nuget: Microsoft.Azure.Cosmos, 3.47.0"

Installed Packages Microsoft.Azure.Cosmos, 3.47.0 Newtonsoft.Json, 13.0.3

In [ ]:
System.String endpoint = "<FILL ME>";
System.String key = "<FILL ME>";

var cosmosClient = new Microsoft.Azure.Cosmos.CosmosClient(endpoint, key);
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("RetailIngest");
Microsoft.Azure.Cosmos.Container container = await database.CreateContainerIfNotExistsAsync("WebsiteMetrics", "/CartID");

Error: System.FormatException: The input is not a valid Base-64 string as it contains a non-base 64 character, more than two padding characters, or an illegal character among the padding characters.
   at System.Convert.FromBase64CharPtr(Char* inputPtr, Int32 inputLength)
   at System.Convert.FromBase64String(String s)
   at Microsoft.Azure.Cosmos.StringHMACSHA256Hash..ctor(String base64EncodedKey)
   at Microsoft.Azure.Cosmos.AuthorizationTokenProviderMasterKey..ctor(String authKey)
   at Microsoft.Azure.Cosmos.AuthorizationTokenProvider.CreateWithResourceTokenOrAuthKey(String authKeyOrResourceToken)
   at Microsoft.Azure.Cosmos.CosmosClient..ctor(String accountEndpoint, String authKeyOrResourceToken, CosmosClientOptions clientOptions)
   at Submission#8.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecu

In [11]:
public class Record
{
  public string id { get; set; }
  public int CartID { get; set; }
  public string Action { get; set; }
  public decimal Price { get; set; }
  public string Country { get; set; }
  public string Item { get; set; }
}

In [ ]:
using System.Text.Json;
using System.IO;

var dataURL = "https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json";
var jsonData = new System.Net.Http.HttpClient().GetStringAsync(dataURL).Result;

Record[] result = JsonSerializer.Deserialize<Record[]>(jsonData);

foreach (Record record in result) {
    await container.UpsertItemAsync<Record>(record, new Microsoft.Azure.Cosmos.PartitionKey(record.CartID)); //43 seconds
}

Error: System.AggregateException: One or more errors occurred. (No such host is known. (cosmosnotebooksdata.blob.core.windows.net:443))
 ---> System.Net.Http.HttpRequestException: No such host is known. (cosmosnotebooksdata.blob.core.windows.net:443)
 ---> System.Net.Sockets.SocketException (11001): No such host is known.
   at System.Net.Sockets.Socket.AwaitableSocketAsyncEventArgs.ThrowException(SocketError error, CancellationToken cancellationToken)
   at System.Net.Sockets.Socket.AwaitableSocketAsyncEventArgs.System.Threading.Tasks.Sources.IValueTaskSource.GetResult(Int16 token)
   at System.Net.Sockets.Socket.<ConnectAsync>g__WaitForConnectWithCancellation|285_0(AwaitableSocketAsyncEventArgs saea, ValueTask connectTask, CancellationToken cancellationToken)
   at System.Net.Http.HttpConnectionPool.ConnectToTcpHostAsync(String host, Int32 port, HttpRequestMessage initialRequest, Boolean async, CancellationToken cancellationToken)
   --- End of inner exception stack trace ---

In [ ]:
System.String queryString = "SELECT c.Action, c.Price, c.Country, c.Item FROM c";
var query = new Microsoft.Azure.Cosmos.QueryDefinition(
    query: queryString
);

Microsoft.Azure.Cosmos.FeedIterator<Record> feed = container.GetItemQueryIterator<Record>(
    queryDefinition: query
);

var results = new System.Collections.Generic.List<Record>();
while (feed.HasMoreResults)
{
    Microsoft.Azure.Cosmos.FeedResponse<Record> response = await feed.ReadNextAsync();
    foreach (Record result in response)
    {
        results.Add(result);
    }
}

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#13.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
var dictionary = new System.Collections.Generic.Dictionary<string, decimal>();

foreach(var result in results)
{
    if (dictionary.ContainsKey(result.Item)) {
        dictionary[result.Item] += result.Price;
    }
    else {
        dictionary.TryAdd (result.Item, result.Price); 

    }
}

dictionary

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)